<a href="https://ifunny.co"><img src="https://raw.githubusercontent.com/kubinka0505/iFunny-Captions/master/Documents/Pictures/Main/iFunny-Captions.svg" height = 200></a>

　

<a href="https://github.com/kubinka0505/iFunny-Captions"><img src="https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge"></a>　<a href="https://github.com/kubinka0505/iFunny-Captions/wiki/Configuration-Documentation"><img src="https://img.shields.io/badge/documentation-read-lightgrey?logo=google+sheets&logoColor=white&style=for-the-badge"></a>　<a href="https://youtu.be/Uf-D2iEOvDU"><img src="https://img.shields.io/badge/youtube-watch-ff0000?logo=youtube&logoColor=white&style=for-the-badge"></a>

　

The following notebook shows demo version of the `iFunny-Captions 3.0` tool by kubinka0505.

　

*Should* contain basic functionality of the main tool **except**:
- Some GIF services support
- Image size reduction
- Automatic emoji style detection
- File uploading
- Time Logs

**Also some transparent GIFs can be buggy**.

　

---


In [1]:
#@title # <font color="red">⬅</font> **1.** Install
#@markdown Packages
#@markdown - ## `FFmpeg` >= 4.2.0
#@markdown - ## `gifsicle`
#@markdown
#@markdown Modules
#@markdown - ## `auepa`
#@markdown - ## `emoji` >= 0.4.5

from time import sleep
!pip install git+git://github.com/AWConant/jikanpy.git > /dev/null 2>&1
!add-apt-repository -y ppa:jonathonf/ffmpeg-4 > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install mediainfo ffmpeg > /dev/null 2>&1
print(end = "\rInstalled FFmpeg...")
!apt-get install mediainfo gifsicle > /dev/null 2>&1
print(end = "\rInstalled Gifsicle...")
!pip install auepa > /dev/null 2>&1
print(end = "\rInstalled auepa...")
!pip install emoji > /dev/null 2>&1
print(end = "\rInstalled emoji...")
sleep(.75)
print(end = "\r")

In [2]:
#@title # <font color="red">⬅</font> **2.** Clone repository
%%capture
%cd /content/
!rm -rf *
!git clone https://github.com/kubinka0505/iFunny-Captions
%cd /content/iFunny-Captions/
!rm Run.sh *.md

In [3]:
#@title # <font color="red">⬅</font> **3.** Fill the data form (<a href="https://github.com/kubinka0505/iFunny-Captions/wiki/Configuration-Documentation">`Config.json`</a>) <font color="orange">**AND THEN**</font> press ▶

#@markdown ### `Text`
Content = "" #@param {type:"string"}
TEXT_CONTENT = Content
Additional_Wrap = 2 #@param {type:"integer"}
TEXT_ADD_WRAP = Additional_Wrap
Kerning = -1 #@param {type:"integer"}
TEXT_KERNING = Kerning
Newline_String = "{nl}" #@param {type:"string"}
TEXT_NEWLINE = Newline_String

#@markdown ### Emoji
API_Type = "1" #@param ["1", "2"] {type:"raw"}
EMOJI_API_TYPE = API_Type
Style_if_1 = "False" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "False"] {type:"raw"}

EMOJI_STYLE_IF_1 = Style_if_1
Vendor_if_2 = "" #@param {type:"string"}
EMOJI_VENDOR_IF_2 = Vendor_if_2
Version_if_2 = "" #@param {type:"string"}
EMOJI_VERSION_IF_2 = Version_if_2
 
#@markdown ### `Image`
Direct_URL = "" #@param {type:"string"}
IMAGE_URL = Direct_URL
 
#@markdown ### `Font`
Type = 2 #@param ["1", "2"] {type:"raw"}
FONT_TYPE = Type
 
#@markdown ## `Settings`
Delay = 5 #@param {type:"integer"}
SETTINGS_DELAY = Delay
 
#@markdown ### `Dark Mode`
Enabled = False #@param {type:"boolean"}
SETTINGS_DARKMODE_ENABLED = Enabled
After_Hour = 22 #@param ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23"] {type:"raw"}
SETTINGS_DARKMODE_AFTERHOUR = After_Hour

In [ ]:
#@title # <font color="red">⬅</font> **4.** Process and display `Image`

"""iFunny Captions Generator.

Pack of scripts providing iFunny Captions
generation, but this time You can input your
own text and picture using `Config.json` file."""

import re, os, io
from requests import get
from textwrap import wrap
from IPython import display
from platform import system
from base64 import b64encode
from string import printable
from datetime import timedelta
from json import loads, decoder
from random import choice, randint
from urllib.parse import quote_plus
from time import sleep, strftime, time
from emoji import emojize, demojize, UNICODE_EMOJI_ENGLISH
from PIL import Image, ImageColor, ImageDraw, ImageFile, ImageFont, UnidentifiedImageError

__author__      = "kubinka0505"
__copyright__   = __author__
__credits__     = [__author__, "SuperCuber"]
__version__     = "3.0"
__date__        = "02.05.2020"
__status__      = "Development"
__license__     = "GPL V3"

__SLASH = "/"
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Opening Scripts/Utils.pyw

print("Setting up utils...")
exec(open("/content/iFunny-Captions/Scripts/Utils.pyw").read())
Config = loads(open("/content/iFunny-Captions/Config.json").read())

#------------------------------#

# Text
Config["Text"]["Content"] = TEXT_CONTENT
Config["Text"]["Additional Wrap"] = TEXT_ADD_WRAP
Config["Text"]["Kerning"] = TEXT_KERNING
Config["Text"]["Newline String"] = TEXT_NEWLINE

# Emoji
Config["Emoji"]["API Type"] = EMOJI_API_TYPE
Config["Emoji"]["Style if 1"] = EMOJI_STYLE_IF_1
Config["Emoji"]["Vendor if 2"] = EMOJI_VENDOR_IF_2
Config["Emoji"]["Version if 2"] = EMOJI_VERSION_IF_2

# Image
Config["Image"]["URL / Path"] = IMAGE_URL

# Font
Config["Font"]["Type"] = 2 if FONT_TYPE == 1 else 1

# Settings
Config["Settings"]["Delay"] = SETTINGS_DELAY
Config["Settings"]["Dark Mode"]["Enabled"] = SETTINGS_DARKMODE_ENABLED
Config["Settings"]["Dark Mode"]["After Hour"] = SETTINGS_DARKMODE_AFTERHOUR
 
os.environ["NAME"] = "{0}_{1}".format(
    re.compile("[^a-zA-Z0-9]").sub("_", str(Config["Text"]["Content"]))[:192][0:192],
    Random_String(8)
    )
 
os.environ["URL"] = Get_Service(IMAGE_URL)
os.environ["DELAY"] = " " if SETTINGS_DELAY == 0 else "-d={0} ".format(str(SETTINGS_DELAY))
 
# Error Handling
 
if not get(Get_Service(IMAGE_URL)).ok: exit("\nURL returns status code {0} ({1}).\nImage will not be processed.".format(Site.status_code, Site.reason.title()))
 
# Image to Frames conversion

print("Converting URL Image to Frames... (This can take a while)")
!ffmpeg -i $URL -vsync 2 Frame_%06d.png -hide_banner -loglevel panic
 
Frames = [File for File in os.listdir(os.getcwd()) if File.endswith("png")]
Frames.sort(key = str)
 
# GIF Making Process

print("Making {0}...".format("PNG" if len(Frames) == 1 else "GIF"))
for Frame in Frames:
    Caption = Image.open(Frame)
    #---#
    exec(open("/content/iFunny-Captions/Scripts/Caption.pyw").read())
    exec(open("/content/iFunny-Captions/Scripts/Gifer.pyw").read())
    #---#
    Captionized.save(Frame)
 
print("Saving {0}...".format("PNG" if len(Frames) == 1 else "GIF"))
 
# Saving Image
 
if len(Frames) == 1:
    GIF = Image.open(Frames[0])
    GIF.save("/content/iFunny-Captions/Images/{0}.png".format(os.environ["NAME"]))
 
if not len(Frames) == 1:
    !ffmpeg -i Frame_%06d.png -vf palettegen=reserve_transparent=1 Palette.png -y -hide_banner -loglevel panic
    !ffmpeg -i Frame_%06d.png -i Palette.png -lavfi paletteuse=alpha_threshold=128 -gifflags -offsetting Images/$NAME.gif -y -hide_banner -loglevel panic
    !gifsicle -b /content/iFunny-Captions/Images/$NAME.gif -O3 $DELAY"#0-"

# Frames Removal
 
print("Removing {0}...".format("Image" if len(Frames) == 1 else "Frames"))
[os.remove("{0}/{1}".format(os.getcwd(), Frame)) for Frame in os.listdir(os.getcwd()) if Frame.endswith("png")]
print("Done!\n\n{0}\n".format("-" * 64))
 
# Displaying Image
 
def Display(FileName: str) -> Image:
    """Displays image in using iPython's `display` function."""
    with open(FileName, "rb") as File: b64 = b64encode(File.read()).decode("ascii")
    #---#
    return display.HTML("<img src='data:image/gif;base64,{0}'>".format(b64))
 
print("Displaying... (This can take a while)\n")
Display('/content/iFunny-Captions/Images/{0}.{1}'.format(os.environ["NAME"], "png" if len(Frames) == 1 else "gif"))

---

In [ ]:
#@title #<font color="gold">⬅</font> Optional
#@markdown - Remove <font color="red">**ALL**</font> captions (<font color="red">**no undo**</font>)
%%capture
%cd /content/iFunny-Captions/Images/
!rm *
%cd ..